In [2]:
'''
Creator : 정지현
Date : 2020-09-25
Description : 2차년도 LSTM 추천 모델 다듬음
Todo : Tensorflow 1.x 버전으로 작성되어서 2.x 버전으로 재작성해야하는데... 후... 하...하하하하라ㅏㄹㅇ하ㅐ라해ㅏㄹㅇ해ㅏㄹㅇ하ㅗㅑ하래ㅗㅇ래ㅑ헝랴ㅐ핼아ㅗ
'''

import pickle

import pandas as pd
import numpy as np
import scipy.sparse as sp

from tqdm import tqdm_notebook as tqdm

import tensorflow as tf

print(tf.__version__)
print('\nInitialized.')

2.5.0

Initialized.


Load already pickled data:

In [3]:
# 데이터 불러오기

with open('bin/df_retail.bin', 'rb') as f_in:
    df = pickle.load(f_in)
    
#df_cust = pd.read_csv('D:/디효니 폴더 ^_^/DeepLearning_2018/2019 2차년도 연구자료/O2ODeepLearning/코드/dataset/진짜_dataset/temp/')

In [4]:
# 데이터 간단한 전처리

df.columns = df.columns.str.lower()
df = df[~df.invoiceno.astype('str').str.startswith('C')].reset_index(drop=True)
df.customerid = df.customerid.fillna(-1).astype('int32')
df = df[df.customerid != -1]

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
# 현재까지는 단순히 각 stockcode를 고유한 index로 변환하는 과정임. 前 파일과 다른 점 없는 듯 함.
item_enc = LabelEncoder()
df.stockcode = item_enc.fit_transform(df.stockcode.astype('str'))
df.stockcode = df.stockcode.astype('int32')
df.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
0,536365,3233,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,2643,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,2847,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,2795,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,2794,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [7]:
user_enc = LabelEncoder() 
df.customerid = user_enc.fit_transform(df.customerid)

Train-test split:

In [8]:
# 데이터 분리
df_train = df[df.invoicedate < '2011-10-09'].reset_index(drop=True)
df_val = df[(df.invoicedate >= '2011-10-09')].reset_index(drop=True)
#df_test = df[df.invoicedate >= '2011-11-09'].reset_index(drop=True)

In [9]:
df_train.shape, df_val.shape#, df_test.shape

((278511, 8), (119413, 8))

In [10]:
# invoiceno를 기준으로 데이터셋의 순서를 바꾸지 않은 상태로 customerid만 unique하게 가져옴

uid_train = df_train.drop_duplicates(subset='invoiceno').customerid.values
uid_val = df_val.drop_duplicates(subset='invoiceno').customerid.values

In [11]:
# 각 거래가 끝나는 시점을 정의
def group_indptr(df):
    indptr, = np.where(df.invoiceno != df.invoiceno.shift())
    indptr = np.append(indptr, len(df)).astype('int32')
    return indptr

indptr_train = group_indptr(df_train)
indptr_val = group_indptr(df_val)

indptr_train

array([     0,      7,      9, ..., 278479, 278510, 278511], dtype=int32)

In [12]:
# df_train에서 각 상품 코드의 출현 수를 카운트 (기준 모델을 생성하기 위함)
from collections import Counter
top_train = Counter(df_train.stockcode)

In [13]:
# 간단한 기준 모델
def baseline(uid, indptr, items, top, k=5):
    n_groups = len(uid)
    n_items = len(items)

    pred_all = np.zeros((n_items, k), dtype=np.int32) # 행 n_items와 열 k로 구성된 0으로 채워진 행렬을 구성함.

    for g in range(n_groups): # n_groups의 수만큼 반복, 행만큼 반복
        t = top.copy()

        start = indptr[g] # 시작 인덱스
        end = indptr[g+1] # 끝 인덱스
        
        for i in range(start, end): # 인덱스 내에서 반복
            pred = [k for (k, c) in t.most_common(5)] # most_common은 입력된 값의 요소들 중 빈도수(frequency)가 높은 순으로 
                                                        #상위 n개를 리스트(list) 안의 투플(tuple) 형태로 반환한다. 
                                                        #n을 입력하지 않은 경우, 요소 전체를 [('값', 개수)]의 형태로 반환한다.
                                                        #pred는 리스트의 형태로 총 5개의 데이터를 저장하게 된다.
            #print(pred)
            pred_all[i] = pred
            #print(pred_all)

            actual = items[i]
            if actual in t: # 추천된 항목 중, 하나를 사용자가 구매했다면 앞으로 만들 추천에서 항목 제거
                del t[actual]

    return pred_all

In [14]:
iid_val = df_val.stockcode.values # validation을 위한 아이템을 저장하기 위한 자료형

In [16]:
pred_baseline = baseline(uid_val, indptr_val, iid_val, top_train)

RNN 기본 모델

전처리 준비

In [19]:
# 각 트랜잭션이 어디에서 시작하고 어디에서 끝나는지를 알려주는 포인터 배열을 재사용함.
def pack_items(users, items_indptr, items_vals):
    n = len(items_indptr) - 1

    result = []
    for i in range(n):
        start = items_indptr[i]
        end = items_indptr[i+1]
        result.append(items_vals[start:end])

    return result

In [20]:
print(len(df_train.stockcode.values))
print(len(indptr_train))

278511
13640


In [21]:
# train set에서 각 사용자가 구매한 상품을 리스트에 저장
train_items = pack_items(indptr_train, indptr_train, df_train.stockcode.values)

df_train_wrap = pd.DataFrame()
df_train_wrap['customerid'] = uid_train
df_train_wrap['items'] = train_items

In [22]:
len(train_items)

13639

In [23]:
df_train_wrap.head()

,customerid,items
0,4017,"[3233, 2643, 2847, 2795, 2794, 1601, 751]"
1,4017,"[1486, 1485]"
2,534,"[3059, 1594, 1597, 1598, 1187, 3114, 1476, 147..."
3,534,"[1801, 1755, 1754, 1756]"
4,534,[769]


### np.pad
파라미터로 지정된 리스트의 양쪽에 0을 채우는 연산

<b>USAGE</b> : pad_width : {sequence, array_like, int}

<b>DESCRIPTION : </b>Number of values padded to the edges of each axis. ((before_1, after_1), … (before_N, after_N)) unique pad widths for each axis. ((before, after),) yields same before and after pad for each axis. (pad,) or int is a shortcut for before = after = pad width for all axes.

<b>EXAMPLE : </b>

a = [1, 2, 3, 4, 5]

np.pad(a, (2, 3), 'constant', constant_values=(4, 6))

array([4, 4, 1, ..., 6, 6, 6])

contant_values를 4와 6, 그리고 pad_width를 2와 3으로 지정했으므로, 리스트의 앞쪽에 4가 2번, 리스트의 뒤쪽에 6이 3번 들어가게 된다.

In [24]:
def pad_seq(data, num_steps):
    data = np.pad(data, pad_width=(1, 0), mode='constant') # 파라미터로 입력된 리스트 data에 대해 앞쪽에 0을 하나 채움.

    n = len(data) # 기존 data의 길이에 +1된 결과를 반환 (0으로 앞쪽이 패딩 되었으므로.)

    if n <= num_steps:
        pad_right = num_steps - n + 1
        data = np.pad(data, pad_width=(0, pad_right), mode='constant') # pad_right의 수만큼 리스트의 오른쪽에 0을 추가함.

    return data

def prepare_train_data(data, num_steps):
    data = pad_seq(data, num_steps)

    X = []
    Y = []

    for i in range(num_steps, len(data)): # i에 들어갈 데이터는 num-steps부터 len(data)까지!
        start = i - num_steps
        X.append(data[start:i])
        Y.append(data[start+1:i+1])

    return X, Y

텐서플로우 모델 만들어보자... 얼마나 힘들게 만들었는데 2버전으로 어케 바꾸지...;;

In [25]:
import tensorflow as tf
rnn = tf.contrib.rnn # rnn 셀

AttributeError: module 'tensorflow' has no attribute 'contrib'

In [77]:
class Config:
    num_steps = 5

    num_items = item_enc.classes_.shape[0] # item 개수
    num_users = user_enc.classes_.shape[0] # user 개수

    init_scale = 0.1
    learning_rate = 1.0 # 학습률
    max_grad_norm = 5
    num_layers = 2
    hidden_size = 200
    embedding_size = 200
    batch_size = 20

config = Config()

### extend 함수와 append 함수
둘 다 리스트에서 활용이 되며, append 함수는 오브젝트를 리스트의 뒤에 포함하는데에 반해, extend는 리스트의 일원으로서 값을 추가함.

<b>예시 : </b>

append()는 object를 맨 뒤에 추가합니다.

    x = [1, 2, 3]

    x.append([4, 5])

    print (x)

    [1, 2, 3, [4, 5]]

extend()는 iterable 객체(리스트, 튜플, 딕셔너리 등)의 엘레멘트를 list에 appending시킵니다.

    x = [1, 2, 3]

    x.extend([4, 5])

    print (x)

    [1, 2, 3, 4, 5]

In [78]:
# 자세한 이론은 p254 참고할 것.

train_items = df_train_wrap['items'] # 각 Customer들이 구매한 상품들을 담은 DataFrame

X_train = []
Y_train = []

for i in range(len(train_items)):
    X, Y = prepare_train_data(train_items[i], config.num_steps) # p254 처럼 만드는 작업
    X_train.extend(X) # append와 비슷하지만, append와는 확연히 다름.
    Y_train.extend(Y)

X_train = np.array(X_train, dtype='int32')
Y_train = np.array(Y_train, dtype='int32')

In [79]:
df_train_wrap[1:]

,customerid,items
1,4017,"[1486, 1485]"
2,534,"[3059, 1594, 1597, 1598, 1187, 3114, 1476, 147..."
3,534,"[1801, 1755, 1754, 1756]"
4,534,[769]
5,189,"[1577, 1576, 1575, 748, 850, 0, 788, 315, 1203..."
...,...,...
13634,1819,"[1435, 1430, 3059, 2033, 2032, 1250]"
13635,4227,"[1474, 2173, 1681, 876, 1950, 2309]"
13636,1045,"[62, 1077, 996, 2583, 1253, 1957, 961, 1254, 2..."
13637,3355,"[162, 2162, 2192, 2312, 2314, 2309, 2308, 2310..."


In [87]:
X_train[3:10], Y_train[3:10]

(array([[   0, 1486, 1485,    0,    0],
        [   0, 3059, 1594, 1597, 1598],
        [3059, 1594, 1597, 1598, 1187],
        [1594, 1597, 1598, 1187, 3114],
        [1597, 1598, 1187, 3114, 1476],
        [1598, 1187, 3114, 1476, 1475],
        [1187, 3114, 1476, 1475,  767]]),
 array([[1486, 1485,    0,    0,    0],
        [3059, 1594, 1597, 1598, 1187],
        [1594, 1597, 1598, 1187, 3114],
        [1597, 1598, 1187, 3114, 1476],
        [1598, 1187, 3114, 1476, 1475],
        [1187, 3114, 1476, 1475,  767],
        [3114, 1476, 1475,  767,  768]]))

Model graph:

In [107]:
# 헬퍼함수 정의....

def lstm_cell(hidden_size, is_training):
    return rnn.BasicLSTMCell(hidden_size, forget_bias=0.0, 
                             state_is_tuple=True, reuse=not is_training)

def rnn_model(inputs, hidden_size, num_layers, batch_size, num_steps, is_training):
    cells = [lstm_cell(hidden_size, is_training) for _ in range(num_layers)] 
    cell = rnn.MultiRNNCell(cells, state_is_tuple=True)

    initial_state = cell.zero_state(batch_size, tf.float32)
    inputs = tf.unstack(inputs, num=num_steps, axis=1) 
                                                        
    outputs, final_state = rnn.static_rnn(cell, inputs, initial_state=initial_state)
    output = tf.reshape(tf.concat(outputs, 1), [-1, hidden_size])

    return output, initial_state, final_state


def model(config, is_training):
    # Model에 사용될 파라미터들을 config 객체로부터 가져옴.
    batch_size = config.batch_size
    num_steps = config.num_steps
    embedding_size = config.embedding_size
    hidden_size = config.hidden_size
    num_items = config.num_items

    # placeholder에서 shape 속성은 입력될 자료형의 크기를 지정하는 것 
    # 아래에서는 [batch_size, num_steps] 만큼의 shape를 입력으로 사용하겠다.
    # 이 그외의 shape가 입력될 경우엔 오류 발생.
    place_x = tf.placeholder(shape=[batch_size, num_steps], dtype=tf.int32)
    place_y = tf.placeholder(shape=[batch_size, num_steps], dtype=tf.int32)

    embedding = tf.get_variable("items", [num_items, embedding_size], dtype=tf.float32)
    inputs = tf.nn.embedding_lookup(embedding, place_x)

    output, initial_state, final_state = \
        rnn_model(inputs, hidden_size, config.num_layers, batch_size, num_steps, is_training)

    W = tf.get_variable("W", [hidden_size, num_items], dtype=tf.float32)
    b = tf.get_variable("b", [num_items], dtype=tf.float32)
    logits = tf.nn.xw_plus_b(output, W, b)
    logits = tf.reshape(logits, [batch_size, num_steps, num_items])

    loss = tf.losses.sparse_softmax_cross_entropy(place_y, logits)
    total_loss = tf.reduce_mean(loss)

    tvars = tf.trainable_variables()
    gradient = tf.gradients(total_loss, tvars)
    clipped, _ = tf.clip_by_global_norm(gradient, config.max_grad_norm)
    optimizer = tf.train.GradientDescentOptimizer(config.learning_rate)

    global_step = tf.train.get_or_create_global_step()
    train_optimizer = optimizer.apply_gradients(zip(clipped, tvars), global_step=global_step)

    result = {}
    result['place_x'] = place_x
    result['place_y'] = place_y
    
    result['initial_state'] = initial_state
    result['final_state'] = final_state
    result['logits'] = logits

    result['total_loss'] = loss
    result['train_op'] = train_optimizer

    return result

모델 초기화랑 학습 ^~^

In [108]:
config = Config()
config_val = Config()
config_val.batch_size = 1
config_val.num_steps = 1

graph = tf.Graph()
graph.seed = 1

with graph.as_default():
    initializer = tf.random_uniform_initializer(-config.init_scale, config.init_scale)

    with tf.name_scope("Train"):
        with tf.variable_scope("Model", reuse=None, initializer=initializer):
            train_model = model(config, is_training=True)

    with tf.name_scope("Valid"):
        with tf.variable_scope("Model", reuse=True, initializer=initializer):
            val_model = model(config_val, is_training=False)

    init = tf.global_variables_initializer()

In [109]:
def prepare_batches(seq, step):
    n = len(seq)
    res = []
    for i in range(0, n, step):
        res.append(seq[i:i+step])
    return res

In [110]:
def run_epoch(session, model, X, Y, batch_size):
    fetches = {
        "total_loss": model['total_loss'],
        "final_state": model['final_state'],
        "eval_op": model['train_op']
    }

    num_steps = X.shape[1]
    all_idx = np.arange(X.shape[0])
    np.random.shuffle(all_idx)
    batches = prepare_batches(all_idx, batch_size)

    initial_state = session.run(model['initial_state'])
    current_state = initial_state

    progress = tqdm(total=len(batches))
    for idx in batches:
        if len(idx) < batch_size:
            continue

        feed_dict = {}
        for i, (c, h) in enumerate(model['initial_state']):
            feed_dict[c] = current_state[i].c
            feed_dict[h] = current_state[i].h

        feed_dict[model['place_x']] = X[idx]
        feed_dict[model['place_y']] = Y[idx]

        vals = session.run(fetches, feed_dict)
        loss = vals["total_loss"]
        current_state = vals["final_state"]

        progress.update(1)
        progress.set_description('%.3f' % loss)
    progress.close()

In [111]:
session = tf.Session(config=None, graph=graph) 
session.run(init)

np.random.seed(0)

run_epoch(session, train_model, X_train, Y_train, batch_size=config.batch_size)

C:\Users\dinb1\Anaconda3\envs\tf1\lib\site-packages\ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


In [112]:
def generate_prediction(uid, indptr, items, model, k):
    n_groups = len(uid)
    n_items = len(items)

    pred_all = np.zeros((n_items, k), dtype=np.int32)
    initial_state = session.run(model['initial_state'])

    fetches = {
        "logits": model['logits'],
        "final_state": model['final_state'],
    }

    for g in tqdm(range(n_groups)):    
        start = indptr[g]
        end = indptr[g+1]

        current_state = initial_state

        feed_dict = {}

        for i, (c, h) in enumerate(model['initial_state']):
            feed_dict[c] = current_state[i].c
            feed_dict[h] = current_state[i].h

        prev = np.array([[0]], dtype=np.int32)

        for i in range(start, end):
            feed_dict[model['place_x']] = prev

            actual = items[i]
            prev[0, 0] = actual

            values = session.run(fetches, feed_dict)
            current_state = values["final_state"]

            logits = values['logits'].reshape(-1)
            pred = np.argpartition(-logits, k)[:k]
            pred_all[i] = pred

    return pred_all

In [113]:
pred_lstm = generate_prediction(uid_val, indptr_val, iid_val, val_model, k=5)

C:\Users\dinb1\Anaconda3\envs\tf1\lib\site-packages\ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


In [116]:
result = ResultClass()
print('Accuracy : %.3f' % result.accuracy_k(iid_val, pred_lstm))
print('Average of Recall : %.3f' % result.compute_recall())

클래스 초기화가 완료되었습니다.
K = 5
전체 추천 성공 개수: 6016
df_val_count 갱신.
Accuracy : 0.050
Average of Recall : 0.052


# 테스트 (밑에서 부터는 안 건드려도 됨)

In [40]:
# 테스트 (성공적! 인지는 모르겠음. 남겨두기!!!! 중요!!!!)
def generate_prediction_user_model(uid, indptr, items, model, k, idx):
    n_groups = 1
    n_items = len(items)

    pred_all = np.zeros(5, dtype=np.int32)
    initial_state = session.run(model['initial_state'])

    fetches = {
        "logits": model['logits'],
        "final_state": model['final_state'],
    }

    for g in tqdm(range(n_groups)):    
        start = indptr[idx]
        end = indptr[idx+1]
        u = uid

        current_state = initial_state

        feed_dict = {}
        feed_dict[model['place_u']] = np.array([[u]], dtype=np.int32)

        for i, (c, h) in enumerate(model['initial_state']):
            feed_dict[c] = current_state[i].c
            feed_dict[h] = current_state[i].h

        prev = np.array([[0]], dtype=np.int32)

        for i in range(start, end):
            feed_dict[model['place_x']] = prev

            actual = items[i]
            prev[0, 0] = actual

            values = session.run(fetches, feed_dict)
            current_state = values["final_state"]

            logits = values['logits'].reshape(-1)
            pred = np.argpartition(-logits, k)[:k]
            #pred_all[i] = pred

    return pred

In [41]:
index = 0
pred_lstm = generate_prediction_user_model(uid_val[index], indptr_val, iid_val, val_model, k=5, idx=index)
pred_lstm

NameError: name 'session' is not defined

In [158]:
saver = tf.train.Saver()
saver.save(session, 'my_test_model')

ValueError: No variables to save

In [61]:
uid_val[0]

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
0,570206,664,DOORMAT SPOTTY HOME SWEET HOME,10,2011-10-09 10:13:00,7.08,1990,United Kingdom
1,570206,2767,DOORMAT FAIRY CAKE,10,2011-10-09 10:13:00,7.08,1990,United Kingdom
2,570206,811,GAOLERS KEYS DECORATIVE GARDEN,2,2011-10-09 10:13:00,3.75,1990,United Kingdom
3,570208,152,EDWARDIAN PARASOL RED,3,2011-10-09 10:23:00,5.95,0,United Kingdom
4,570208,21,EDWARDIAN PARASOL BLACK,3,2011-10-09 10:23:00,5.95,0,United Kingdom
...,...,...,...,...,...,...,...,...
154146,581587,1527,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,222,France
154147,581587,1803,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,222,France
154148,581587,2145,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,222,France
154149,581587,2146,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,222,France


In [42]:
result = ResultClass()
print('Accuracy : %.3f' % result.accuracy_k(iid_val, pred_lstm))
print('Precision : %.3f' % result.compute_accuracy())
print('Average of Recall : %.3f' % result.compute_recall())

클래스 초기화가 완료되었습니다.
K = 5
전체 추천 성공 개수: 33946
df_val_count 갱신.
Accuracy : 0.220
Precision : 0.776
Average of Recall : 0.249


In [97]:
acc, result_pre = accuracy_k(iid_val, pred_lstm)#, result_pre = accuracy_k(iid_val, pred_lstm)
#result_pre = compute_accuracy(5)
#result_rec = compute_recall()
#result_f1 = (2*result_pre*result_rec)/(result_pre+result_rec)
print('Accuracy : %.3f' % acc)
print('Precision : %.3f' % result_pre)
#print('Recall : %.3f' % result_rec)
#print('F1 : %.3f' % result_f1)

Accuracy : 0.332
Precision : 0.038


In [37]:
df_val.tail()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
154146,581587,1527,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,222,France
154147,581587,1803,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,222,France
154148,581587,2145,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,222,France
154149,581587,2146,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,222,France
154150,581587,1093,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,222,France


In [34]:
iid_val

array([ 664, 2767,  811, ..., 2145, 2146, 1093])

In [38]:
# Test
with open('./bin/lstm_k-5.bin', 'rb') as f_in:
    check_pred = pickle.load(f_in)

In [40]:
len(check_pred)

154151

In [43]:
pred_lstm

array([[3507, 3508,  173, 1317, 1336],
       [3508, 3507, 1317,  173, 2096],
       [ 816, 1381, 1348,  817, 3301],
       ...,
       [1573, 2102, 2101, 1314,  184],
       [1573, 1314, 2102,  184, 2101],
       [ 932, 1093, 1094, 1531, 1471]])

In [23]:
df_val.to_csv('./datasets/df_val.csv', index=False)